In [1]:
import os
import dotenv
import langchain_neo4j
import langchain_huggingface

# import time
# import neo4j

from IPython import display

from langchain import chains
import langchain_google_genai
import langchain_ollama

# from langchain.chains import RetrievalQA  # Deprecated, use `create_retrieval_chain` instead
# from langchain.chains import RetrievalQAWithSourcesChain
# from langchain_google_genai import ChatGoogleGenerativeAI

# https://python.langchain.com/docs/versions/migrating_chains/retrieval_qa/

dotenv.load_dotenv(".env")

True

In [2]:
embeddings = langchain_huggingface.HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

d:\Kuliah Semester 7\Docling\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
text = "Apa kewajiban prosesor data pribadi?"
query_result = embeddings.embed_query(text)

# show only the first 100 characters of the stringified vector
print(str(query_result)[:100] + "...")
# query_result

[-0.016636021435260773, 0.07482326775789261, -0.09838518500328064, -0.06209137290716171, -0.12543356...


In [4]:
doc_result = embeddings.embed_documents([text, "This is not a test document."])
for result in doc_result:
    print(str(result)[:100] + "...")

[-0.01663600467145443, 0.07482324540615082, -0.09838520735502243, -0.06209135428071022, -0.125433579...
[-0.018281754106283188, 0.11222248524427414, -0.014233257621526718, 0.023113159462809563, 0.01809122...


You can also customize responses by using a custom Cypher snippet that can fetch other information from the graph. Under the hood, the final Cypher statement is constructed like so:

```python
read_query = (
  "CALL db.index.vector.queryNodes($index, $k, $embedding) "
  "YIELD node, score "
) + retrieval_query
```

The retrieval query must return the following three columns:
- `text`: Union[str, Dict] = Value used to populate `page_content` of a document
- `score`: Float = Similarity score
- `metadata`: Dict = Additional metadata of a document


In [5]:
# NODE_VECTOR_INDEX_QUERY = (
#     "CALL db.index.vector.queryNodes"
#     "($vector_index_name, $top_k * $effective_search_ratio, $query_vector) "
#     "YIELD node, score "
#     "WITH node, score LIMIT $top_k"
# ) + retrieval_query

# retrieval_query = """
# RETURN node {.text} AS text, score, {id: node.id} AS metadata
# """

# CARA DI BAWAH INI MENGAMBIL SEMUA NODE AWAL + NODE HASIL RELATED_TO DAN REFER_TO,
# LALU MENGURUTKANNYA DARI SKOR COSINE SIMILARITY TERBESAR KE TERKECIL,
# DAN TERAKHIR HASILNYA DI-LIMIT HANYA 10 ATAU 15

# TODO: AKU KEPIKIRAN GINI, NODE AWAL + NODE REFER_TO HARUS DIAMBIL SEMUA.
# JIKA TOTAL AKHIRNYA KURANG DARI 10 ATAU 15, MAKA SISANYA DIAMBIL DARI NODE RELATED_TO.
# YANG DIURUTKAN DARI SKOR COSINE SIMILARITY TERBESAR KE TERKECIL,
# LALU DIAMBIL SEBANYAK JUMLAH SISA NODE YANG KURANG

# CARANYA BISA PAKAI 1 CYPHER PANJANG, ATAU BEBERAPA CYPHER.
# JIKA PAKAI BEBERAPA CYPHER, MAKA SUDAH PASTI AKU HARUS MEMBUAT RETRIEVER INI
# SEBAGAI SEBUAH NODE SENDIRI DI LANGGRAPH. JADI TIDAK BISA PAKAI chains.RetrievalQA.from_chain_type
# SEPERTI CONTOH DI BAWAH

# ATAU PAKAI CARA RETRIEVER SEKARANG INI BISA AJA SEBENARNYA SUDAH CUKUP

retrieval_query = """
WITH collect(node) AS init_nodes
UNWIND init_nodes AS init_node

// 2. Cari node yang terhubung melalui RELATED_TO atau REFER_TO
OPTIONAL MATCH (init_node)-[:RELATED_TO|REFER_TO]-(related_node)
WITH init_nodes, collect(DISTINCT related_node) AS related_nodes

// 3. Gabungkan hasil init_nodes + related_nodes
WITH init_nodes + related_nodes AS all_nodes
UNWIND all_nodes AS candidate_node
WITH DISTINCT candidate_node
// WHERE candidate_node IS NOT NULL

// 4. Hitung ulang cosine similarity (jika fungsi tersedia)
WITH candidate_node, vector.similarity.cosine(
    $query_vector, candidate_node.embedding
) AS similarity_score
ORDER BY similarity_score DESC
LIMIT $limit

RETURN 
    candidate_node {.text} AS text,
    similarity_score AS score,
    candidate_node {.id, .name} AS metadata
"""

import neo4j_vector_mod
from neo4j_graphrag.types import HybridSearchRanker


db_1 = neo4j_vector_mod.Neo4jVectorMod.from_existing_graph(
    embedding=embeddings,
    node_label="Effective",
    text_node_properties=["text"],
    embedding_node_property="embedding",
    index_name="effective_vector_index",
    keyword_index_name="effective_fulltext_index",  # Fulltext Index
    search_type="hybrid",                           # Fulltext + Vector
    # retrieval_query=retrieval_query,                # Custom retrieval query
    
    url=os.environ["DATABASE_HOST"],
    username=os.environ["DATABASE_USERNAME"],
    password=os.environ["DATABASE_PASSWORD"],
    database=os.environ["DATABASE_SMALL"],
)

query = "Apa definisi dari prosesor data pribadi?"
docs_with_score = db_1.similarity_search_with_score(query, k=5, params={"limit": 15}, ranker=HybridSearchRanker.NAIVE)
docs_with_score
# Pasal 29, Pasal 31, Pasal 35, Pasal 36, Pasal 37, Pasal 38, dan Pasal 39

[(Document(metadata={'chapter': '6', 'paragraph': '', 'name': 'UU No. 27 Tahun 2022 Pasal 52', 'number': '52', 'part': '3'}, page_content='\ntext: Undang-undang (UU) Nomor 27 Tahun 2022 tentang Pelindungan Data Pribadi, BAB VI - KEWAJIBAN PENGENDALI DATA PRIBADI DAN PROSESOR DATA PRIBADI DALAM PEMROSESAN DATA PRIBADI, Bagian Ketiga - Kewajiban Prosesor Data Pribadi, Pasal 52:\nKetentuan mengenai kewajiban Pengendali Data Pribadi sebagaimana dimaksud dalam Pasal 29, Pasal 31, Pasal 35, Pasal 36, Pasal 37, Pasal 38, dan Pasal 39 berlaku juga terhadap Prosesor Data Pribadi.'),
  1.0),
 (Document(metadata={'chapter': '6', 'paragraph': '', 'name': 'UU No. 27 Tahun 2022 Pasal 51', 'number': '51', 'part': '3'}, page_content='\ntext: Undang-undang (UU) Nomor 27 Tahun 2022 tentang Pelindungan Data Pribadi, BAB VI - KEWAJIBAN PENGENDALI DATA PRIBADI DAN PROSESOR DATA PRIBADI DALAM PEMROSESAN DATA PRIBADI, Bagian Ketiga - Kewajiban Prosesor Data Pribadi, Pasal 51:\n(1) Dalam hal Pengendali Data P

In [7]:
retrieval_query = """
WITH collect(node) AS init_nodes
UNWIND init_nodes AS init_node

// 2. Cari node yang terhubung melalui RELATED_TO atau REFER_TO
OPTIONAL MATCH (init_node)-[:RELATED_TO|REFER_TO]-(related_node)
WITH init_nodes, collect(DISTINCT related_node) AS related_nodes

// 3. Gabungkan hasil init_nodes + related_nodes
WITH init_nodes + related_nodes AS all_nodes
UNWIND all_nodes AS candidate_node
WITH DISTINCT candidate_node
// WHERE candidate_node IS NOT NULL

// 4. Hitung ulang cosine similarity (jika fungsi tersedia)
WITH candidate_node, vector.similarity.cosine(
    $query_vector, candidate_node.embedding
) AS similarity_score
ORDER BY similarity_score DESC
LIMIT $limit

RETURN 
    candidate_node {.text} AS text,
    similarity_score AS score,
    candidate_node {.id, .name} AS metadata
"""

import neo4j_vector_mod


db_1 = neo4j_vector_mod.Neo4jVectorMod.from_existing_graph(
    embedding=embeddings,
    node_label="Effective",
    text_node_properties=["text"],
    embedding_node_property="embedding",
    index_name="effective_vector_index",
    keyword_index_name="effective_fulltext_index",  # Fulltext Index
    search_type="hybrid",                           # Fulltext + Vector
    # retrieval_query=retrieval_query,                # Custom retrieval query
    
    url=os.environ["DATABASE_HOST"],
    username=os.environ["DATABASE_USERNAME"],
    password=os.environ["DATABASE_PASSWORD"],
    database=os.environ["DATABASE_SMALL"],
)

query = "Apa definisi dari prosesor data pribadi?"
docs_with_score = db_1.similarity_search_with_score(query, k=5, params={"limit": 15}, ranker=HybridSearchRanker.LINEAR, alpha=0.5)
docs_with_score
# Pasal 29, Pasal 31, Pasal 35, Pasal 36, Pasal 37, Pasal 38, dan Pasal 39

[(Document(metadata={'chapter': '6', 'paragraph': '', 'name': 'UU No. 27 Tahun 2022 Pasal 52', 'number': '52', 'part': '3'}, page_content='\ntext: Undang-undang (UU) Nomor 27 Tahun 2022 tentang Pelindungan Data Pribadi, BAB VI - KEWAJIBAN PENGENDALI DATA PRIBADI DAN PROSESOR DATA PRIBADI DALAM PEMROSESAN DATA PRIBADI, Bagian Ketiga - Kewajiban Prosesor Data Pribadi, Pasal 52:\nKetentuan mengenai kewajiban Pengendali Data Pribadi sebagaimana dimaksud dalam Pasal 29, Pasal 31, Pasal 35, Pasal 36, Pasal 37, Pasal 38, dan Pasal 39 berlaku juga terhadap Prosesor Data Pribadi.'),
  0.9690952616079891),
 (Document(metadata={'chapter': '6', 'paragraph': '', 'name': 'UU No. 27 Tahun 2022 Pasal 51', 'number': '51', 'part': '3'}, page_content='\ntext: Undang-undang (UU) Nomor 27 Tahun 2022 tentang Pelindungan Data Pribadi, BAB VI - KEWAJIBAN PENGENDALI DATA PRIBADI DAN PROSESOR DATA PRIBADI DALAM PEMROSESAN DATA PRIBADI, Bagian Ketiga - Kewajiban Prosesor Data Pribadi, Pasal 51:\n(1) Dalam hal Pe

In [9]:
# NODE_VECTOR_INDEX_QUERY = (
#     "CALL db.index.vector.queryNodes"
#     "($vector_index_name, $top_k * $effective_search_ratio, $query_vector) "
#     "YIELD node, score "
#     "WITH node, score LIMIT $top_k"
# ) + retrieval_query

# retrieval_query = """
# RETURN node {.text} AS text, score, {id: node.id} AS metadata
# """

# CARA DI BAWAH INI MENGAMBIL SEMUA NODE AWAL + NODE HASIL RELATED_TO DAN REFER_TO,
# LALU MENGURUTKANNYA DARI SKOR COSINE SIMILARITY TERBESAR KE TERKECIL,
# DAN TERAKHIR HASILNYA DI-LIMIT HANYA 10 ATAU 15

# TODO: AKU KEPIKIRAN GINI, NODE AWAL + NODE REFER_TO HARUS DIAMBIL SEMUA.
# JIKA TOTAL AKHIRNYA KURANG DARI 10 ATAU 15, MAKA SISANYA DIAMBIL DARI NODE RELATED_TO.
# YANG DIURUTKAN DARI SKOR COSINE SIMILARITY TERBESAR KE TERKECIL,
# LALU DIAMBIL SEBANYAK JUMLAH SISA NODE YANG KURANG

# CARANYA BISA PAKAI 1 CYPHER PANJANG, ATAU BEBERAPA CYPHER.
# JIKA PAKAI BEBERAPA CYPHER, MAKA SUDAH PASTI AKU HARUS MEMBUAT RETRIEVER INI
# SEBAGAI SEBUAH NODE SENDIRI DI LANGGRAPH. JADI TIDAK BISA PAKAI chains.RetrievalQA.from_chain_type
# SEPERTI CONTOH DI BAWAH

# ATAU PAKAI CARA RETRIEVER SEKARANG INI BISA AJA SEBENARNYA SUDAH CUKUP

retrieval_query = """
// CALL db.index.vector.queryNodes("effective_vector_index", 5, $query_vector)
// YIELD node AS node, score AS mainScore

WITH collect(node) AS init_nodes 
UNWIND init_nodes AS init

OPTIONAL MATCH (init)-[:REFER_TO]-(refer_node)
OPTIONAL MATCH (init)-[:RELATED_TO]-(related_node)

WITH init_nodes,
     [n IN collect(DISTINCT refer_node) WHERE NOT n IN init_nodes] AS refer_nodes,
     [n IN collect(DISTINCT related_node) WHERE NOT n IN init_nodes] AS related_nodes

RETURN init_nodes, refer_nodes, related_nodes
"""


db_2 = langchain_neo4j.Neo4jVector.from_existing_graph(
    embedding=embeddings,
    node_label="Effective",
    text_node_properties=["text"],
    embedding_node_property="embedding",
    index_name="effective_vector_index",
    keyword_index_name="effective_fulltext_index",  # Fulltext Index
    search_type="hybrid",                           # Fulltext + Vector
    retrieval_query=retrieval_query,                # Custom retrieval query
    
    url=os.environ["DATABASE_HOST"],
    username=os.environ["DATABASE_USERNAME"],
    password=os.environ["DATABASE_PASSWORD"],
    database=os.environ["DATABASE_SMALL"],
)

query = "Apa kewajiban prosesor data pribadi?"
docs_with_score = db_2.similarity_search_with_score(query, k=5)
docs_with_score
# Pasal 29, Pasal 31, Pasal 35, Pasal 36, Pasal 37, Pasal 38, dan Pasal 39

KeyError: 'text'

In [20]:
db_3 = langchain_neo4j.Neo4jVector.from_existing_graph(
    embedding=embeddings,
    node_label="Effective",
    text_node_properties=["text"],
    embedding_node_property="embedding",
    index_name="effective_vector_index",
    keyword_index_name="effective_fulltext_index",  # Fulltext Index
    search_type="hybrid",                           # Fulltext + Vector
    # retrieval_query=retrieval_query,                # Custom retrieval query
    
    url=os.environ["DATABASE_HOST"],
    username=os.environ["DATABASE_USERNAME"],
    password=os.environ["DATABASE_PASSWORD"],
    database=os.environ["DATABASE_SMALL"],
)

query = "Apa kewajiban prosesor data pribadi?"
docs_with_score = db_3.similarity_search_with_score(query, k=10)
display(docs_with_score)
# Pasal 29, Pasal 31, Pasal 35, Pasal 36, Pasal 37, Pasal 38, dan Pasal 39

[(Document(metadata={'chapter': '6', 'paragraph': '', 'name': 'UU No. 27 Tahun 2022 Pasal 52', 'number': '52', 'part': '3'}, page_content='\ntext: Undang-undang (UU) Nomor 27 Tahun 2022 tentang Pelindungan Data Pribadi, BAB VI - KEWAJIBAN PENGENDALI DATA PRIBADI DAN PROSESOR DATA PRIBADI DALAM PEMROSESAN DATA PRIBADI, Bagian Ketiga - Kewajiban Prosesor Data Pribadi, Pasal 52:\nKetentuan mengenai kewajiban Pengendali Data Pribadi sebagaimana dimaksud dalam Pasal 29, Pasal 31, Pasal 35, Pasal 36, Pasal 37, Pasal 38, dan Pasal 39 berlaku juga terhadap Prosesor Data Pribadi.'),
  1.0),
 (Document(metadata={'chapter': '6', 'paragraph': '', 'name': 'UU No. 27 Tahun 2022 Pasal 51', 'number': '51', 'part': '3'}, page_content='\ntext: Undang-undang (UU) Nomor 27 Tahun 2022 tentang Pelindungan Data Pribadi, BAB VI - KEWAJIBAN PENGENDALI DATA PRIBADI DAN PROSESOR DATA PRIBADI DALAM PEMROSESAN DATA PRIBADI, Bagian Ketiga - Kewajiban Prosesor Data Pribadi, Pasal 51:\n(1) Dalam hal Pengendali Data P

In [11]:
# for doc, score in docs_with_score:
#     print("-" * 80)
#     print("score: ", score)
#     print("metadata:", doc.metadata)
#     print(doc.page_content.strip())
#     print("-" * 80)

In [12]:
# URI = os.environ["DATABASE_HOST"]
# USERNAME = os.environ["DATABASE_USERNAME"]
# PASSWORD = os.environ["DATABASE_PASSWORD"]
# DATABASE = os.environ["DATABASE_SMALL"]

# # https://neo4j.com/docs/genai/tutorials/embeddings-vector-indexes/query/search-prompt/
# with neo4j.GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD)) as driver:
#     driver.verify_connectivity()
    
#     model = sentence_transformers.SentenceTransformer('all-MiniLM-L6-v2')

#     start_time = time.time()
#     query_prompt = 'Apa kewajiban penyelenggara sistem elektronik?'
#     # DEFINISI MENGGANGGU, AKU MAU NYA AMBIL PASAL? GIMANA SOLUSINYA?
#     query_embedding = model.encode(query_prompt)

#     retrieval_propotition = {
#         "effective_vector_index": 10,
#         "definition_vector_index": 5
#     }

#     all_related_text = []
#     print(f'Text that relate to`{query_prompt}`:')
#     for vector_index_name, amount in retrieval_propotition.items():
#         related_text, _, _ = driver.execute_query(
#             query_='''
#             CALL db.index.vector.queryNodes($vector_index_name, $amount, $query_embedding)
#             YIELD node, score
#             RETURN node.number AS number, score, labels(node) AS labels, node.text AS text
#             ''',
#             database_=DATABASE,
#             parameters_= {
#                 "vector_index_name": vector_index_name,
#                 "amount": amount,
#                 "query_embedding": query_embedding,
#             }
#         )
        
#         for record in related_text:
#             all_related_text.append(record)
#             print(record)
    
#     print(f'Duration: {round(time.time() - start_time, 3)} seconds')

In [75]:
retriever = db_1.as_retriever(search_kwargs={"k": 5, "params": {"limit": 15}, "ranker": HybridSearchRanker.LINEAR, "alpha": 0.5})
retriever.invoke(query)[0]

chain = chains.RetrievalQA.from_chain_type(
    # langchain_ollama.ChatOllama(model="llama3.1:8b-instruct-q4_0", temperature=0.1),
    # langchain_ollama.ChatOllama(model="gemma3:4b-it-q4_K_M", temperature=0.1),
    langchain_google_genai.ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        temperature=0.1,
        api_key=os.environ["GOOGLE_API_KEY"]
    ),
    chain_type="stuff",
    retriever=retriever
)

# Vector
response = chain.invoke(
    {"query": "Apa kewajiban dari prosesor data pribadi?"},
    return_only_outputs=True,
)["result"]

display.display(display.Markdown(response))

C:\Users\Bayu Siddhi\AppData\Local\Temp\ipykernel_21864\2809359912.py:180: DeprecationWarning: The default returned 'id' field in the search results will be removed. Please switch to using 'elementId' instead.
  read_query, filter_params = get_search_query(
C:\Users\Bayu Siddhi\AppData\Local\Temp\ipykernel_21864\2809359912.py:180: DeprecationWarning: The default returned 'id' field in the search results will be removed. Please switch to using 'elementId' instead.
  read_query, filter_params = get_search_query(


Berdasarkan Undang-Undang Nomor 27 Tahun 2022 tentang Pelindungan Data Pribadi, kewajiban Prosesor Data Pribadi meliputi:

1.  Melakukan pemrosesan Data Pribadi berdasarkan perintah Pengendali Data Pribadi (Pasal 51 ayat 1).
2.  Pemrosesan Data Pribadi dilaksanakan sesuai ketentuan yang diatur dalam Undang-Undang ini (Pasal 51 ayat 2).
3.  Wajib mendapatkan persetujuan tertulis dari Pengendali Data Pribadi sebelum melibatkan Prosesor Data Pribadi lain (Pasal 51 ayat 5).
4.  Ketentuan mengenai kewajiban Pengendali Data Pribadi sebagaimana dimaksud dalam Pasal 29, Pasal 31, Pasal 35, Pasal 36, Pasal 37, Pasal 38, dan Pasal 39 berlaku juga terhadap Prosesor Data Pribadi (Pasal 52). Ini berarti Prosesor Data Pribadi juga wajib:

    *   Memastikan akurasi, kelengkapan, dan konsistensi Data Pribadi (Pasal 29).
    *   Melakukan perekaman terhadap seluruh kegiatan pemrosesan Data Pribadi (Pasal 31).
    *   Melindungi dan memastikan keamanan Data Pribadi yang diprosesnya (Pasal 35).
    *   Menjaga kerahasiaan Data Pribadi (Pasal 36).
    *   Melakukan pengawasan terhadap setiap pihak yang terlibat dalam pemrosesan Data Pribadi di bawah kendalinya (Pasal 37).
    *   Melindungi Data Pribadi dari pemrosesan yang tidak sah (Pasal 38).
    *   Mencegah Data Pribadi diakses secara tidak sah (Pasal 39).
5.  Wajib melaksanakan perintah lembaga dalam rangka penyelenggaraan Pelindungan Data Pribadi sesuai dengan Undang-Undang ini (Pasal 49).

In [ ]:
retriever = db_2.as_retriever()
retriever.invoke(query)[0]

chain = chains.RetrievalQA.from_chain_type(
    # langchain_ollama.ChatOllama(model="llama3.1:8b-instruct-q4_0", temperature=0.1),
    # langchain_ollama.ChatOllama(model="gemma3:4b-it-q4_K_M", temperature=0.1),
    langchain_google_genai.ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        temperature=0.0,
        api_key=os.environ["GOOGLE_API_KEY"]
    ),
    chain_type="stuff",
    retriever=retriever
)

# Vector
response = chain.invoke(
    {"query": "Apa kewajiban prosesor data pribadi?"},
    return_only_outputs=True,
)["result"]

display.display(display.Markdown(response))

In [30]:
retriever = db_3.as_retriever()
retriever.invoke(query)[0]

chain = chains.RetrievalQA.from_chain_type(
    # langchain_ollama.ChatOllama(model="llama3.1:8b-instruct-q4_0", temperature=0.1),
    # langchain_ollama.ChatOllama(model="gemma3:4b-it-q4_K_M", temperature=0.1),
    langchain_google_genai.ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        temperature=0.0,
        api_key=os.environ["GOOGLE_API_KEY"]
    ),
    chain_type="stuff",
    retriever=retriever
)

# Vector
response = chain.invoke(
    {"query": "Apa kewajiban prosesor data pribadi?"},
    return_only_outputs=True,
)["result"]

display.display(display.Markdown(response))

Kewajiban Prosesor Data Pribadi tercantum dalam UU Nomor 27 Tahun 2022 tentang Pelindungan Data Pribadi, BAB VI, Bagian Ketiga, Pasal 51 dan 52.

Pasal 51 menjelaskan bahwa:
1. Prosesor Data Pribadi wajib melakukan pemrosesan Data Pribadi berdasarkan perintah Pengendali Data Pribadi.
2. Pemrosesan Data Pribadi dilaksanakan sesuai ketentuan yang diatur dalam Undang-Undang ini.
3. Pemrosesan Data Pribadi termasuk dalam tanggung jawab Pengendali Data Pribadi.
4. Prosesor Data Pribadi dapat melibatkan Prosesor Data Pribadi lain dalam melakukan pemrosesan Data Pribadi.
5. Prosesor Data Pribadi wajib mendapatkan persetujuan tertulis dan Pengendali Data Pribadi sebelum melibatkan Prosesor Data Pribadi lain.
6. Dalam hal Prosesor Data Pribadi melakukan pemrosesan Data Pribadi di luar perintah dan tujuan yang ditetapkan Pengendali Data Pribadi, pemrosesan Data Pribadi menjadi tanggung jawab Prosesor Data Pribadi.

Pasal 52 menjelaskan bahwa ketentuan mengenai kewajiban Pengendali Data Pribadi sebagaimana dimaksud dalam Pasal 29, Pasal 31, Pasal 35, Pasal 36, Pasal 37, Pasal 38, dan Pasal 39 berlaku juga terhadap Prosesor Data Pribadi.

Selain itu, Pengendali Data Pribadi dan/atau Prosesor Data Pribadi wajib melaksanakan perintah lembaga dalam rangka penyelenggaraan Pelindungan Data Pribadi sesuai dengan Undang-Undang ini (Pasal 49).

In [67]:
# Hybrid (Llama)
response = chain.invoke(
    {"query": "Apa kewajiban penyelenggara sistem elektronik?"},
    return_only_outputs=True,
)["result"]

display.display(display.Markdown(response))

Menurut Peraturan Pemerintah (PP) Nomor 71 Tahun 2019 tentang Penyelenggaraan Sistem dan Transaksi Elektronik, kewajiban penyelenggara sistem elektronik antara lain:

1. Menerapkan manajemen risiko terhadap kerusakan atau kerugian yang ditimbulkan (Pasal 12).
2. Melakukan edukasi kepada pengguna sistem elektronik tentang hak, kewajiban, dan tanggung jawab seluruh pihak terkait, serta prosedur pengajuan komplain (Pasal 28).
3. Melakukan uji kelaikan sistem elektronik terhadap seluruh komponen atau sebagian komponen dalam sistem elektronik sesuai dengan karakteristik kebutuhan pelindungan dan sifat strategis penyelenggaraan sistem elektronik (Pasal 34).

Selain itu, juga tidak boleh melakukan perbuatan yang dilarang menurut Undang-undang (UU) Nomor 11 Tahun 2008 tentang Informasi dan Transaksi Elektronik, seperti mengakses komputer dan/atau sistem elektronik milik orang lain dengan cara apa pun tanpa hak atau melawan hukum.

In [70]:
# Hybrid (Gemma)
response = chain.invoke(
    {"query": "Apa kewajiban penyelenggara sistem elektronik?"},
    return_only_outputs=True,
)["result"]

display.display(display.Markdown(response))

Berdasarkan teks yang diberikan, berikut adalah kewajiban penyelenggara sistem elektronik:

*   **Menerapkan manajemen risiko:** (PP Nomor 71 Tahun 2019, Pasal 12)
*   **Melakukan edukasi kepada Pengguna Sistem Elektronik:** (PP Nomor 71 Tahun 2019, Pasal 28) - Edukasi ini mencakup hak, kewajiban, tanggung jawab, dan prosedur pengajuan komplain.
*   **Melakukan Uji Kelayakan Sistem Elektronik:** (PP Nomor 71 Tahun 2019, Pasal 34) - Uji ini dapat dilakukan secara menyeluruh atau sebagian sesuai dengan kebutuhan pelindungan dan sifat strategis penyelenggaraan.

Selain itu, penyelenggara sistem elektronik juga dilarang mengakses komputer dan/atau sistem elektronik milik orang lain tanpa hak atau melawan hukum (UU Nomor 11 Tahun 2008, Pasal 30).